In [2]:
from dataclasses import dataclass , field
from typing import Literal
@dataclass
class ModelDataKargs:
    storage_type : Literal['mem' , 'disk'] = 'mem'
    beg_date : int = 20170101
    end_date : int = 20991231
    interval : int = 120
    tra_model : bool = False
    skip_horizon : int = 20
    input_span : int = 2400
    buffer_param : dict = field(default_factory=dict)
    weight_scheme: dict = field(default_factory=dict)
    num_output : int = 1
    train_ratio : float = 0.8
    sample_method : Literal['total_shuffle' , 'sequential' , 'both_shuffle' , 'train_shuffle'] = 'sequential'
    batch_size : int = 10000
    input_step_day : int = 5
    test_step_day : int = 1
    verbosity : int = 5

a = ModelDataKargs()

In [5]:
a.__dict__.keys()

dict_keys(['storage_type', 'beg_date', 'end_date', 'interval', 'tra_model', 'skip_horizon', 'input_span', 'buffer_param', 'weight_scheme', 'num_output', 'train_ratio', 'sample_method', 'batch_size', 'input_step_day', 'test_step_day', 'verbosity'])

In [5]:
from src.util.loader import Storage

In [1]:
from scripts.util.data.ModelData import *
import os
model_data = ModelData('day' , if_train = False)

Use device name: NVIDIA GeForce RTX 4090
Load  2 DataBlocks ...... cost 0.11 secs
Align 2 DataBlocks ...... cost 0.10 secs


In [12]:
model_path = './model/resnet_gru_30m'
model_num  = 0
model_param = torch.load(f'{model_path}/model_params.pt')[model_num]
model_file = sorted([p for p in os.listdir(f'{model_path}/{model_num}') if p.endswith('swalast.pt')])[-1]
model_date = int(model_file.split('.')[0])
model = torch.load(f'{model_path}/{model_num}/{model_file}')

In [3]:
dataloader_param = model_data.get_dataloader_param('test' , 'test' , model_date=model_date , param=model_param)   
model_data.create_dataloader(*dataloader_param)

Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


In [11]:
assert len(model_data.dataloaders['test']) , len(model_data.dataloaders['test']) 
batch_data = model_data.dataloaders['test'][-1]
date  = model_data.model_test_dates[-1]
secid = len(model_data.index[0][batch_data['i'][:,0].cpu().numpy()])
pred = model(batch_data['x'])

5055

In [14]:
import logging
a = logging.getLogger()


In [16]:
type(a)

logging.RootLogger

In [8]:
model_data.dataloaders['test'].iterator

['./data/minibatch/test/test.0.pt',
 './data/minibatch/test/test.1.pt',
 './data/minibatch/test/test.2.pt',
 './data/minibatch/test/test.3.pt',
 './data/minibatch/test/test.4.pt',
 './data/minibatch/test/test.5.pt',
 './data/minibatch/test/test.6.pt',
 './data/minibatch/test/test.7.pt',
 './data/minibatch/test/test.8.pt',
 './data/minibatch/test/test.9.pt',
 './data/minibatch/test/test.10.pt',
 './data/minibatch/test/test.11.pt',
 './data/minibatch/test/test.12.pt',
 './data/minibatch/test/test.13.pt',
 './data/minibatch/test/test.14.pt',
 './data/minibatch/test/test.15.pt',
 './data/minibatch/test/test.16.pt',
 './data/minibatch/test/test.17.pt',
 './data/minibatch/test/test.18.pt',
 './data/minibatch/test/test.19.pt',
 './data/minibatch/test/test.20.pt',
 './data/minibatch/test/test.21.pt',
 './data/minibatch/test/test.22.pt',
 './data/minibatch/test/test.23.pt',
 './data/minibatch/test/test.24.pt',
 './data/minibatch/test/test.25.pt',
 './data/minibatch/test/test.26.pt',
 './data/mi